# Machine Learning Enhanced Shannon's Demon

This notebook demonstrates how to enhance Shannon's Demon with machine learning using AlphaPy Pro.

**📚 Learning Path**: This is Part 3 of the tutorial series:
- **01_basic_demo.ipynb** - Basic concepts
- **02_real_data.ipynb** - Real market data analysis
- **03_ml_enhanced.ipynb** ← You are here (ML-enhanced strategies)
- **../demon.ipynb** - Comprehensive analysis (must-see!)

**🎯 Quick Start**: For AlphaPy integration, run:
```bash
python prepare_data.py --symbol BTC-USD
alphapy --config config/model.yml
```

## 1. Setup and Integration with AlphaPy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add AlphaPy to path
sys.path.append('../../..')
sys.path.append('..')

# Import AlphaPy modules
from alphapy.data import load_data
from alphapy.features import create_features
from alphapy.model import Model
from alphapy.utilities import get_pandas_data_frame

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Feature Engineering for Rebalancing Signals

In [ ]:
def create_rebalancing_features(data, lookback_periods=[5, 10, 20, 50]):
    """
    Create features for predicting optimal rebalancing times.
    """
    features = pd.DataFrame(index=data.index)
    
    # Price-based features
    for period in lookback_periods:
        features[f'returns_{period}d'] = data['Close'].pct_change(period)
        features[f'volatility_{period}d'] = data['Close'].pct_change().rolling(period).std()
        features[f'volume_ratio_{period}d'] = data['Volume'] / data['Volume'].rolling(period).mean()
    
    # Technical indicators
    features['rsi_14'] = calculate_rsi(data['Close'], 14)
    features['macd_signal'] = calculate_macd_signal(data['Close'])
    features['bollinger_position'] = calculate_bollinger_position(data['Close'])
    features['atr_14'] = calculate_atr(data, 14)
    
    # Portfolio-specific features
    features['weight_deviation'] = calculate_weight_deviation(data['Close'])
    features['rebalance_profit_potential'] = estimate_rebalance_profit(features['weight_deviation'], 
                                                                      features['volatility_20d'])
    
    # Market regime features
    features['trend_strength'] = calculate_trend_strength(data['Close'])
    features['market_regime'] = identify_market_regime(data['Close'])
    
    # Time-based features
    features['day_of_week'] = data.index.dayofweek
    features['day_of_month'] = data.index.day
    features['month'] = data.index.month
    
    return features

In [ ]:
# Technical indicator functions
def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index."""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd_signal(prices, fast=12, slow=26, signal=9):
    """Calculate MACD signal."""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    return macd - macd_signal

def calculate_bollinger_position(prices, period=20, num_std=2):
    """Calculate position within Bollinger Bands."""
    sma = prices.rolling(period).mean()
    std = prices.rolling(period).std()
    upper_band = sma + (std * num_std)
    lower_band = sma - (std * num_std)
    position = (prices - lower_band) / (upper_band - lower_band)
    return position

def calculate_atr(data, period=14):
    """Calculate Average True Range."""
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    atr = true_range.rolling(period).mean()
    return atr

def calculate_weight_deviation(prices, target_weight=0.5):
    """Calculate deviation from target portfolio weight."""
    # Simulate portfolio weights
    returns = prices.pct_change().fillna(0)
    portfolio_value = 10000
    asset_value = portfolio_value * target_weight
    cash_value = portfolio_value * (1 - target_weight)
    
    weights = []
    for r in returns:
        asset_value *= (1 + r)
        total_value = asset_value + cash_value
        weight = asset_value / total_value
        weights.append(weight)
    
    return pd.Series(weights, index=prices.index) - target_weight

def estimate_rebalance_profit(weight_deviation, volatility, transaction_cost=0.001):
    """Estimate potential profit from rebalancing."""
    # Simplified profit estimation
    potential_profit = np.abs(weight_deviation) * volatility * 2
    cost = transaction_cost * 2  # Buy and sell
    return potential_profit - cost

def calculate_trend_strength(prices, period=20):
    """Calculate trend strength using linear regression."""
    def lin_reg_slope(y):
        x = np.arange(len(y))
        slope, _ = np.polyfit(x, y, 1)
        return slope
    
    slopes = prices.rolling(period).apply(lin_reg_slope, raw=True)
    return slopes / prices.rolling(period).std()

def identify_market_regime(prices, short_period=50, long_period=200):
    """Identify market regime (trending, ranging, volatile)."""
    sma_short = prices.rolling(short_period).mean()
    sma_long = prices.rolling(long_period).mean()
    volatility = prices.pct_change().rolling(20).std()
    
    regime = pd.Series(index=prices.index, dtype=float)
    
    # Trending up
    regime[(sma_short > sma_long) & (volatility < volatility.median())] = 1
    
    # Trending down
    regime[(sma_short < sma_long) & (volatility < volatility.median())] = -1
    
    # Volatile
    regime[volatility > volatility.quantile(0.75)] = 2
    
    # Ranging
    regime.fillna(0, inplace=True)
    
    return regime

## 3. Create Training Data with Labeled Rebalancing Signals

In [ ]:
def create_rebalancing_labels(data, threshold=0.2, min_profit=0.002):
    """
    Create labels for optimal rebalancing points.
    
    1 = Should rebalance
    0 = Should not rebalance
    """
    prices = data['Close']
    
    # Calculate portfolio simulation
    portfolio_value = 10000
    target_weight = 0.5
    asset_value = portfolio_value * target_weight
    cash_value = portfolio_value * (1 - target_weight)
    n_shares = asset_value / prices.iloc[0]
    
    labels = []
    last_rebalance = 0
    
    for i, price in enumerate(prices):
        # Current portfolio state
        asset_value = n_shares * price
        total_value = asset_value + cash_value
        current_weight = asset_value / total_value
        
        # Check rebalancing conditions
        weight_deviation = abs(current_weight - target_weight)
        time_since_rebalance = i - last_rebalance
        
        # Estimate profit from rebalancing
        rebalance_value = abs(total_value * (current_weight - target_weight))
        transaction_cost = rebalance_value * 0.001
        
        # Look ahead to see if rebalancing would be profitable
        if i < len(prices) - 20:  # Need future data to determine profitability
            future_returns = prices.iloc[i:i+20].pct_change().std() * np.sqrt(252)
            expected_profit = weight_deviation * future_returns * total_value * 0.1
            
            should_rebalance = (
                weight_deviation > threshold and 
                expected_profit > transaction_cost * 2 and
                time_since_rebalance > 5
            )
        else:
            should_rebalance = False
        
        labels.append(1 if should_rebalance else 0)
        
        # Update portfolio if rebalanced
        if should_rebalance:
            # Rebalance
            target_asset_value = total_value * target_weight
            n_shares = (target_asset_value - transaction_cost) / price
            asset_value = n_shares * price
            cash_value = total_value - asset_value - transaction_cost
            last_rebalance = i
    
    return pd.Series(labels, index=data.index)

In [ ]:
# Load data and create features
from pathlib import Path
data_path = Path('../data/BTC-USD.csv')

# Load Bitcoin data
btc_data = pd.read_csv(data_path)
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data.set_index('Date', inplace=True)
btc_data = btc_data['2018-01-01':'2023-12-31']

# Create features
features = create_rebalancing_features(btc_data)

# Create labels
labels = create_rebalancing_labels(btc_data)

# Combine features and labels
ml_data = features.copy()
ml_data['rebalance_signal'] = labels

# Remove NaN values
ml_data = ml_data.dropna()

print(f"Dataset shape: {ml_data.shape}")
print(f"Positive labels (rebalance): {(ml_data['rebalance_signal'] == 1).sum()}")
print(f"Negative labels (hold): {(ml_data['rebalance_signal'] == 0).sum()}")
print(f"Class balance: {(ml_data['rebalance_signal'] == 1).sum() / len(ml_data) * 100:.2f}%")

## 4. Train Machine Learning Models

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb

# Prepare data for training
X = ml_data.drop('rebalance_signal', axis=1)
y = ml_data['rebalance_signal']

# Split data (time-based split for financial data)
split_date = '2022-01-01'
X_train = X[X.index < split_date]
y_train = y[y.index < split_date]
X_test = X[X.index >= split_date]
y_test = y[y.index >= split_date]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Features: {X.shape[1]}")

In [ ]:
# Train multiple models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, max_depth=5, random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=100, max_depth=5, random_state=42, verbose=-1)
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Store results
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'probabilities': y_pred_proba,
        'report': classification_report(y_test, y_pred, output_dict=True)
    }
    
    # Print classification report
    print(classification_report(y_test, y_pred))

## 5. Feature Importance Analysis

In [ ]:
# Get feature importance from best model
best_model_name = max(results.keys(), key=lambda x: results[x]['report']['accuracy'])
best_model = results[best_model_name]['model']

# Get feature importances
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    # Plot top features
    plt.figure(figsize=(10, 8))
    top_features = feature_importance.head(15)
    plt.barh(top_features['feature'], top_features['importance'])
    plt.xlabel('Importance')
    plt.title(f'Top 15 Features - {best_model_name}')
    plt.tight_layout()
    plt.show()
    
    print("Top 10 Most Important Features:")
    print("=" * 50)
    print(feature_importance.head(10).to_string(index=False))

## 6. Backtest ML-Enhanced Strategy

In [ ]:
class MLEnhancedShannonsDemon:
    """
    Shannon's Demon enhanced with machine learning predictions.
    """
    
    def __init__(self, model, initial_capital=10000, target_allocation=0.5,
                 ml_confidence_threshold=0.7, transaction_cost=0.001):
        
        self.model = model
        self.initial_capital = initial_capital
        self.target_allocation = target_allocation
        self.ml_confidence_threshold = ml_confidence_threshold
        self.transaction_cost = transaction_cost
        
        # Results storage
        self.portfolio_values = []
        self.allocations = []
        self.trades = []
        self.ml_signals = []
    
    def run(self, data, features):
        """
        Run ML-enhanced strategy.
        """
        prices = data['Close'].values
        dates = data.index
        
        # Initialize portfolio
        risky_value = self.initial_capital * self.target_allocation
        safe_value = self.initial_capital * (1 - self.target_allocation)
        n_shares = risky_value / prices[0]
        
        # Run strategy
        for i in range(len(prices)):
            # Current values
            risky_value = n_shares * prices[i]
            total_value = risky_value + safe_value
            current_allocation = risky_value / total_value
            
            # Store values
            self.portfolio_values.append(total_value)
            self.allocations.append(current_allocation)
            
            # Get ML prediction
            if dates[i] in features.index:
                feature_row = features.loc[[dates[i]]]
                ml_prob = self.model.predict_proba(feature_row)[0, 1]
                ml_signal = ml_prob > self.ml_confidence_threshold
                self.ml_signals.append(ml_prob)
                
                # Rebalance if ML says so
                if ml_signal:
                    # Calculate rebalancing trade
                    target_risky_value = total_value * self.target_allocation
                    risky_trade_value = target_risky_value - risky_value
                    shares_traded = risky_trade_value / prices[i]
                    
                    # Apply transaction costs
                    cost = abs(risky_trade_value) * self.transaction_cost
                    
                    # Update portfolio
                    n_shares += shares_traded
                    risky_value = n_shares * prices[i]
                    safe_value = total_value - risky_value - cost
                    
                    # Record trade
                    self.trades.append({
                        'date': dates[i],
                        'price': prices[i],
                        'ml_confidence': ml_prob,
                        'shares_traded': shares_traded,
                        'value_traded': risky_trade_value,
                        'cost': cost
                    })
            else:
                self.ml_signals.append(0)
        
        return self

In [ ]:
# Backtest on test period
test_data = btc_data[btc_data.index >= split_date]
test_features = features[features.index >= split_date]

# Run different strategies
strategies_results = {}

# Buy and Hold
initial = 10000
btc_shares = (initial * 0.5) / test_data['Close'].iloc[0]
bh_values = btc_shares * test_data['Close'].values + (initial * 0.5)
strategies_results['Buy and Hold'] = bh_values

# Traditional Shannon's Demon (threshold-based)
from collections import namedtuple
SDResult = namedtuple('SDResult', ['portfolio_values', 'trades'])

def run_traditional_sd(data, threshold=0.2):
    prices = data['Close'].values
    portfolio_values = []
    trades = []
    
    # Initialize
    risky_value = initial * 0.5
    safe_value = initial * 0.5
    n_shares = risky_value / prices[0]
    
    for i, price in enumerate(prices):
        risky_value = n_shares * price
        total_value = risky_value + safe_value
        portfolio_values.append(total_value)
        
        current_allocation = risky_value / total_value
        if abs(current_allocation - 0.5) > threshold:
            # Rebalance
            target_risky = total_value * 0.5
            trade_value = target_risky - risky_value
            n_shares += trade_value / price
            cost = abs(trade_value) * 0.001
            safe_value = total_value * 0.5 - cost
            trades.append(i)
    
    return SDResult(portfolio_values, trades)

trad_sd = run_traditional_sd(test_data)
strategies_results['Traditional SD'] = trad_sd.portfolio_values

# ML-Enhanced strategies
for name, model_data in results.items():
    ml_sd = MLEnhancedShannonsDemon(model_data['model'])
    ml_sd.run(test_data, test_features)
    strategies_results[f'ML-{name}'] = ml_sd.portfolio_values

# Plot comparison
plt.figure(figsize=(14, 8))
for name, values in strategies_results.items():
    plt.plot(test_data.index[:len(values)], values, label=name, linewidth=2)

plt.title('Strategy Performance Comparison')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Performance Analysis

In [ ]:
# Calculate performance metrics for all strategies
def calculate_strategy_metrics(values, periods_per_year=252):
    returns = pd.Series(values).pct_change().dropna()
    
    total_return = (values[-1] / values[0] - 1) * 100
    annual_return = ((values[-1] / values[0]) ** (periods_per_year / len(values)) - 1) * 100
    volatility = returns.std() * np.sqrt(periods_per_year) * 100
    sharpe = (annual_return - 2) / volatility if volatility > 0 else 0
    
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = ((cumulative - running_max) / running_max).min() * 100
    
    return {
        'Total Return (%)': total_return,
        'Annual Return (%)': annual_return,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe,
        'Max Drawdown (%)': drawdown
    }

# Calculate metrics for all strategies
metrics_comparison = pd.DataFrame()
for name, values in strategies_results.items():
    metrics = calculate_strategy_metrics(values)
    metrics_comparison[name] = metrics

metrics_comparison = metrics_comparison.T
print("Performance Metrics Comparison:")
print("=" * 100)
print(metrics_comparison.round(2).to_string())

In [ ]:
# Visualize metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Annual Return
ax = axes[0, 0]
metrics_comparison['Annual Return (%)'].plot(kind='bar', ax=ax, color='green')
ax.set_title('Annual Return Comparison')
ax.set_ylabel('Annual Return (%)')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Sharpe Ratio
ax = axes[0, 1]
metrics_comparison['Sharpe Ratio'].plot(kind='bar', ax=ax, color='blue')
ax.set_title('Sharpe Ratio Comparison')
ax.set_ylabel('Sharpe Ratio')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Volatility
ax = axes[1, 0]
metrics_comparison['Volatility (%)'].plot(kind='bar', ax=ax, color='orange')
ax.set_title('Volatility Comparison')
ax.set_ylabel('Volatility (%)')

# Max Drawdown
ax = axes[1, 1]
metrics_comparison['Max Drawdown (%)'].plot(kind='bar', ax=ax, color='red')
ax.set_title('Maximum Drawdown Comparison')
ax.set_ylabel('Max Drawdown (%)')

plt.tight_layout()
plt.show()

## 8. ML Signal Analysis

In [ ]:
# Analyze ML predictions
best_ml_name = f'ML-{best_model_name}'
ml_sd = MLEnhancedShannonsDemon(best_model)
ml_sd.run(test_data, test_features)

# Create analysis DataFrame
ml_analysis = pd.DataFrame({
    'date': test_data.index[:len(ml_sd.ml_signals)],
    'price': test_data['Close'].values[:len(ml_sd.ml_signals)],
    'ml_confidence': ml_sd.ml_signals,
    'allocation': ml_sd.allocations
})

# Add trade markers
trade_dates = [t['date'] for t in ml_sd.trades]
ml_analysis['traded'] = ml_analysis['date'].isin(trade_dates)

# Plot ML signals
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Price and trades
ax = axes[0]
ax.plot(ml_analysis['date'], ml_analysis['price'], label='BTC Price')
trade_points = ml_analysis[ml_analysis['traded']]
ax.scatter(trade_points['date'], trade_points['price'], color='red', s=50, zorder=5, label='ML Trades')
ax.set_ylabel('Price ($)')
ax.set_title('Bitcoin Price and ML-Triggered Trades')
ax.legend()
ax.grid(True, alpha=0.3)

# ML Confidence
ax = axes[1]
ax.plot(ml_analysis['date'], ml_analysis['ml_confidence'], label='ML Confidence', color='green')
ax.axhline(0.7, color='red', linestyle='--', label='Threshold')
ax.fill_between(ml_analysis['date'], 0, ml_analysis['ml_confidence'], alpha=0.3, color='green')
ax.set_ylabel('ML Confidence')
ax.set_title('Machine Learning Rebalancing Confidence')
ax.legend()
ax.grid(True, alpha=0.3)

# Portfolio Allocation
ax = axes[2]
ax.plot(ml_analysis['date'], ml_analysis['allocation'] * 100, label='BTC Allocation')
ax.axhline(50, color='red', linestyle='--', label='Target')
ax.fill_between(ml_analysis['date'], 40, 60, alpha=0.2, color='gray', label='Target Range')
ax.set_ylabel('BTC Allocation (%)')
ax.set_xlabel('Date')
ax.set_title('Portfolio Allocation Over Time')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Trade statistics
print(f"\nML Trading Statistics:")
print("=" * 50)
print(f"Total trades: {len(ml_sd.trades)}")
print(f"Average trades per month: {len(ml_sd.trades) / (len(test_data) / 30):.2f}")
print(f"Average ML confidence when trading: {np.mean([t['ml_confidence'] for t in ml_sd.trades]):.3f}")
print(f"Total transaction costs: ${sum(t['cost'] for t in ml_sd.trades):.2f}")

## 9. Advanced ML Features: Ensemble and Time-Series Models

In [ ]:
# Create ensemble predictions
ensemble_predictions = np.zeros(len(X_test))
for name, model_data in results.items():
    ensemble_predictions += model_data['probabilities']

ensemble_predictions /= len(results)

# Create ensemble strategy
class EnsembleModel:
    def __init__(self, models):
        self.models = models
    
    def predict_proba(self, X):
        predictions = np.zeros((len(X), 2))
        for model in self.models.values():
            predictions += model['model'].predict_proba(X)
        return predictions / len(self.models)

ensemble_model = EnsembleModel(results)
ensemble_sd = MLEnhancedShannonsDemon(ensemble_model, ml_confidence_threshold=0.6)
ensemble_sd.run(test_data, test_features)

# Add to results
strategies_results['ML-Ensemble'] = ensemble_sd.portfolio_values

# Recalculate metrics with ensemble
ensemble_metrics = calculate_strategy_metrics(ensemble_sd.portfolio_values)
print("\nEnsemble Strategy Performance:")
print("=" * 50)
for metric, value in ensemble_metrics.items():
    print(f"{metric}: {value:.2f}")

## 10. Conclusions and Production Considerations

In [ ]:
# Summary visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Final values comparison
final_values = {name: values[-1] for name, values in strategies_results.items()}
sorted_strategies = sorted(final_values.items(), key=lambda x: x[1], reverse=True)

ax1.bar(range(len(sorted_strategies)), [v for _, v in sorted_strategies])
ax1.set_xticks(range(len(sorted_strategies)))
ax1.set_xticklabels([n for n, _ in sorted_strategies], rotation=45, ha='right')
ax1.set_title('Final Portfolio Values')
ax1.set_ylabel('Portfolio Value ($)')
ax1.grid(True, alpha=0.3)

# Risk-return scatter
returns = []
volatilities = []
names = []

for name, values in strategies_results.items():
    metrics = calculate_strategy_metrics(values)
    returns.append(metrics['Annual Return (%)'])
    volatilities.append(metrics['Volatility (%)'])
    names.append(name)

ax2.scatter(volatilities, returns, s=100)
for i, name in enumerate(names):
    ax2.annotate(name, (volatilities[i], returns[i]), xytext=(5, 5), 
                textcoords='offset points', fontsize=8)

ax2.set_xlabel('Volatility (%)')
ax2.set_ylabel('Annual Return (%)')
ax2.set_title('Risk-Return Profile')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Key Findings and Recommendations

### 1. Machine Learning Impact
- ML models can successfully predict profitable rebalancing opportunities
- Ensemble approaches often provide more stable predictions
- Feature importance shows volatility and weight deviation are key indicators

### 2. Best Practices for Production
- **Data Quality**: Ensure real-time data feeds are reliable
- **Model Updates**: Retrain models periodically (monthly/quarterly)
- **Risk Management**: Set maximum position limits and drawdown controls
- **Transaction Costs**: Account for slippage and market impact

### 3. Further Enhancements
- **Deep Learning**: LSTM/GRU for time-series prediction
- **Reinforcement Learning**: Learn optimal rebalancing policy
- **Multi-Asset**: Extend to portfolios with 3+ assets
- **Market Regime**: Adapt strategy based on market conditions

### 4. Implementation Checklist
- [ ] Set up real-time data pipeline
- [ ] Implement model serving infrastructure
- [ ] Create monitoring dashboard
- [ ] Add alerting for anomalies
- [ ] Implement position sizing logic
- [ ] Add transaction cost modeling
- [ ] Create backtesting framework
- [ ] Set up model retraining pipeline

### 5. Risk Warnings
- Past performance doesn't guarantee future results
- ML models can fail in unprecedented market conditions
- Always use proper risk management
- Start with small capital for live testing